### Домашняя работа к Уроку 2
### Студент: Абрамов А.В.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [13]:
def weighted_random_recommendation(items, items_weights, n=5):
    """Случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items)
    items_weights = np.array(items_weights)
    recs = np.random.choice(items, n, replace=False, p = items_weights)
    
    return recs.tolist()

In [6]:
data_train['probability'] = 1 / len(data_train)

C:\Users\User\AppData\Local\Temp\ipykernel_11392\3098316510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['probability'] = 1 / len(data_train)


In [7]:
%%time
items = data_train.item_id.unique()
items_weights = data_train.groupby(['item_id']).sum(['probability'])

CPU times: total: 375 ms
Wall time: 377 ms


In [8]:
items_weights = items_weights['probability']

In [9]:
items_weights

item_id
25671       1.316661e-06
26081       4.388872e-07
26093       4.388872e-07
26190       4.388872e-07
26355       4.388872e-07
                ...     
17381856    4.388872e-07
17382205    4.388872e-07
17383227    4.388872e-07
17827644    4.388872e-07
17829232    8.777743e-07
Name: probability, Length: 86865, dtype: float64

In [14]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, items_weights, n=5))

In [15]:
result.head()

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423729, 1087442, 2046588, 12132576, 2047752]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5567704, 13512334, 12604177, 828501, 8181556]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8157495, 1717051, 7410169, 984705, 8090815]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[952550, 6552171, 956493, 12456940, 1094878]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5592557, 6773142, 12430972, 899029, 9681214]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [26]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [27]:
def precision_at_k(result_recommendation, actual_recommendation, k=5):
    result_recommendation = result_recommendation[:k]
    precision = len(set(result_recommendation).intersection(actual_recommendation))/k
    return precision

In [41]:
precision_at_k(result['random_recommendation'][45], result['actual'][34])

0.0

In [28]:
# Преобразуем данные в столбце 'actual' от списка со строкой к нормальному списку
result['actual'] = result['actual'].apply(lambda x: [int(i) for i in x.replace('[', '').replace(']', '').split()])

In [29]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [46]:
result.apply(lambda row: precision_at_k(eval(row['random_recommendation']), row['actual']), axis=1).mean()

0.0005876591576885406

In [47]:
result.apply(lambda row: precision_at_k(eval(row['popular_recommendation']), row['actual']), axis=1).mean()

0.15523996082272082

In [48]:
result.apply(lambda row: precision_at_k(eval(row['itemitem']), row['actual']), axis=1).mean()

0.033594515181195064

In [49]:
result.apply(lambda row: precision_at_k(eval(row['cosine']), row['actual']), axis=1).mean()

0.03525954946131262

In [50]:
result.apply(lambda row: precision_at_k(eval(row['tfidf']), row['actual']), axis=1).mean()

0.03614103819784544

In [52]:
result.apply(lambda row: precision_at_k(eval(row['own_purchases']), row['actual']), axis=1).mean()

0.17992164544563902

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code